In [ ]:
from PyKDL import Rotation
from urdf_parser_py.urdf import URDF
from pykdl_utils.kdl_kinematics import KDLKinematics

import numpy as np
import math
from scipy import interpolate
from scipy.optimize import leastsq, least_squares

import lcm
from bot_core import *

In [ ]:
# https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles#Euler_Angles_from_Quaternion
def quat_to_euler(q):
    qq = q[1]*q[2] + q[3]*q[0]
    if abs(qq)>(0.5-0.05):
        print "qq",qq
    phi = math.atan2( (2*(q[0]*q[1]+q[2]*q[3])) , (1-2*(q[1]**2 + q[2]**2)) )
    theta = math.asin( 2*(q[0]*q[2]-q[3]*q[1]))
    psi = math.atan2( (2*(q[0]*q[3] + q[1]*q[2])) , (1-2*(q[2]**2 + q[3]**2)) )
    return phi, theta, psi

In [ ]:
def get_joint_map(names,values):
    jmap = {}
    for i in range(len(names)):
        jmap[names[i]] = values[i]
    return jmap

In [ ]:
def joints_from_lcm(file_path, chain_joints):

    joints = []

    log = lcm.EventLog(file_path, 'r')
    for event in log:
        if event.channel == "EST_ROBOT_STATE":
            v = []
            rs = robot_state_t.decode(event.data)
            v.append(rs.utime)
            jmap = get_joint_map(rs.joint_name, rs.joint_position)

            for j in chain_joints:
                v.append(jmap[j])
            joints.append(v)

    return joints

In [ ]:
def pose_from_lcm(file_path):

    poses = []

    log = lcm.EventLog(file_path, 'r')
    for event in log:
        if event.channel == "VICON_VAL_HAND_POSE":
            pose = rigid_transform_t.decode(event.data)
            [x,y,z] = pose.trans
            # Euler
            [phi, theta, psi] = quat_to_euler(pose.quat)
            poses.append([pose.utime,x,y,z,phi,theta,psi])
            # quaternion
#             [qw,qx,qy,qz] = pose.quat
#             poses.append([pose.utime,x,y,z,qw,qx,qy,qz])

    return poses

In [ ]:
# interpolate
# use ref as the reference timestamps, e.g. the timestamps of the final synchronized signal
def interp(ref, target):
    t = target[:,0]
    p = target[:,1:]
    p_sync = []
    for i in range(p.shape[1]):
        f = interpolate.interp1d(t, p[:,i], axis=0)
        sync = f(ref)
        p_sync.append(sync)
    return ref, np.array(p_sync).T

In [ ]:
def pose_diff(q_offset, fk, q_rep, true_pose):
    q_new = q_rep + q_offset
    pose = kdl_kin.forward(q_new)
    fk_rot = Rotation(pose[0,0], pose[0,1], pose[0,2],
                     pose[1,0], pose[1,1], pose[1,2],
                     pose[2,0], pose[2,1], pose[2,2])
    # Euler
    [r,p,y] = fk_rot.GetRPY()
    fk_pose = np.array([pose[0:3,3].T.tolist()[0], [r, p, y]]).flatten()
    
    # quaternion
#     [x,y,z,w] = fk_rot.GetQuaternion() # note different notation
#     p = [pose[0:3,3].T.tolist()[0], [w,x,y,z]]
#     fk_pose = [item for sublist in p for item in sublist]
    
    err = np.linalg.norm(true_pose-fk_pose)
    #print "err",err
    #err = true_pose-fk_pose
    return err

In [ ]:
# error over all synchronized samples
def obj_funct(q_offset, joints_sync, poses_sync, fk):
    assert(joints_sync.shape[0]==poses_sync.shape[0])
    sum_err = 0
    for i in range(joints_sync.shape[0]):
        q_rep = joints_sync[i,:]
        true_pose = poses_sync[i,:]
        err = pose_diff(q_offset, fk, q_rep, true_pose)
        sum_err += err
    return sum_err

In [ ]:
robot = URDF.from_xml_file("/home/christian/Development/oh-distro-private/software/models/val_description/urdf/valkyrie_sim.urdf")
kdl_kin = KDLKinematics(robot, "pelvis", "leftPalm")

In [ ]:
chain_joints = kdl_kin.get_joint_names()
print "joints in chain:",chain_joints

In [ ]:
joints = joints_from_lcm("/home/christian/Downloads/logs/20160727_cr-hand-movement-with-vicon-marker/vicon-arm_movement.lcmlog", chain_joints)
joints = np.array(joints)
print "joints dim:",joints.shape

In [ ]:
#poses = pose_from_lcm("/home/christian/Downloads/logs/dart_vicon_comp/hand_poses/out_arm_vicon_nop.lcmlog")
poses = pose_from_lcm("/home/christian/Downloads/logs/dart_vicon_comp/out_arm_vicon_nop.lcmlog")
poses = np.array(poses)
print "pose dim:",poses.shape

In [ ]:
poses[0,:]

In [ ]:
tref = poses[:-1,0] # ignore last element
print "ref min max",min(tref), max(tref)
print "tar min max",min(joints[:,0]), max(joints[:,0])
print min(tref) < min(joints[:,0])
print max(tref) > max(joints[:,0])

In [ ]:
# synchronize data
t_sync, joints_sync = interp(tref, joints)
poses_sync = poses[:-1,1:]

In [ ]:
print joints_sync.shape
print t_sync.shape
print len(t_sync)
print poses_sync.shape

In [ ]:
q_init = np.zeros((len(chain_joints)))
#print q_init.shape
opt_offset = least_squares(fun=obj_funct, x0=q_init, args=(joints_sync, poses_sync, kdl_kin))

In [ ]:
opt_offset

In [ ]:
opt_offset.x

In [ ]:
chain_joints

In [ ]:
obj_funct(opt_offset.x, joints_sync, poses_sync, kdl_kin)

In [ ]:
obj_funct(q_init, joints_sync, poses_sync, kdl_kin)